In [1]:
!pip show roboflow ultralytics


Name: roboflow
Version: 1.1.58
Summary: Official Python package for working with the Roboflow API
Home-page: https://github.com/roboflow-ai/roboflow-python
Author: Roboflow
Author-email: support@roboflow.com
License: UNKNOWN
Location: /home/andreuet/PIA_Projectes/Projecte_02/matricules_venv/lib/python3.12/site-packages
Requires: certifi, cycler, filetype, idna, kiwisolver, matplotlib, numpy, opencv-python-headless, Pillow, pillow-heif, python-dateutil, python-dotenv, PyYAML, requests, requests-toolbelt, six, tqdm, urllib3
Required-by: 
---
Name: ultralytics
Version: 8.3.97
Summary: Ultralytics YOLO 🚀 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: 
Author: 
Author-email: Glenn Jocher <glenn.jocher@ultralytics.com>, Jing Qiu <jing.qiu@ultralytics.com>
License: AGPL-3.0
Location: /home/andreuet/PIA_Projectes/Projecte_02/matricules_venv/lib/python3.12/site-packages
Requires: matplotlib, numpy, opencv-python, pan

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="5HYzrdPm6LO8xmXDVn3G")
project = rf.workspace("vc").project("matriculas-espanolas")
version = project.version(1)
dataset = version.download("yolov11")

                

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
import os
from ultralytics import YOLO

# Load a COCO-pretrained YOLO11n model
model = YOLO("yolo11n.pt")


results = model.train(data=os.path.join(dataset.location, "data.yaml"), epochs=50, imgsz=640)



In [2]:
model = YOLO("./yolo11n.pt")

NameError: name 'YOLO' is not defined

In [4]:
images=[
    "./Matriculas-Españolas-1/valid/images/1668448029089_jpg.rf.779ea16b2d2e47a624e8ebfa0d3e96a9.jpg"
]

results = model.predict(
    source=images,
    # conf=0.25,    # Umbral de confianza
    save=False,   # No que dibuje YOLO; lo haremos manualmente con OpenCV
    verbose=False
)

In [ ]:
import cv2
import numpy as np

# Suponiendo que 'results' contiene las predicciones de YOLO
# y 'images' contiene la ruta de la imagen

# Cargar la imagen
image_path = images[0]
image = cv2.imread(image_path)

# Verificar que la imagen se cargó correctamente
if image is None:
    print(f"No se pudo cargar la imagen: {image_path}")
    exit()

# Obtener las predicciones para esta imagen (asumiendo que solo hay una imagen)
result = results[0]

# Dibujar cada bounding box
for box in result.boxes:
    # Obtener coordenadas de la bounding box (x1, y1, x2, y2)
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    
    # Obtener la confianza y la clase
    confidence = box.conf[0]
    class_id = int(box.cls[0])
    class_name = result.names[class_id]
    
    # Color para la bounding box (aquí usamos verde)
    color = (0, 255, 0)
    
    # Dibujar el rectángulo
    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    
    # Crear texto con la clase y confianza
    label = f"{class_name}: {confidence:.2f}"
    
    # Tamaño del texto
    (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    
    # Dibujar fondo para el texto
    cv2.rectangle(image, (x1, y1 - text_height - 5), (x1 + text_width, y1), color, -1)
    
    # Dibujar el texto
    cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

# Mostrar la imagen
cv2.imshow("Detections", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Opcional: guardar la imagen con las detecciones
# cv2.imwrite("detections.jpg", image)